In [1]:
import xarray as xr
import os
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn import metrics

res = 2.8125
local_dir = f"E:\\weatherbench\\{res}deg"

In [2]:
def get_all_4_corners():
    d = xr.open_dataset("E:\\weatherbench\\2.8125deg\\2m_temperature\\2m_temperature_2.8125deg\\2m_temperature_1979_2.8125deg.nc")
    lats = [lt.item() for lt in d['lat']]
    lons = [ln.item() for ln in d['lon']]
    output = []
    for i, lat in enumerate(lats):
        for j, lon in enumerate(lons):
            if i == len(lats)-1:
                continue
            if j == len(lons)-1:
                continue
            output.append([(lat, lon), (lat, lons[j+1]), (lats[i+1], lon), (lats[i+1], lons[j+1])])
    return output


def get_daily_value_for_year(feature_info, lat, lon, year):
    feature, df_fld = feature_info
    d = xr.open_dataset(os.path.join(local_dir, feature, f"{feature}_{res}deg", f"{feature}_{year}_{res}deg.nc"))
    data = d.sel(lon=lon, lat=lat).to_dataframe()[df_fld]
    # hourly temperature: each day covers 24 entries. Take the daily average over 24hrs
    one_year_daily=np.array([np.mean(data[i:i+24]) for i in range(0, len(data), 24)])
    return one_year_daily[:365].reshape((1, 365))


def linearize(x):
    linearized = np.array([])
    for chunk in x:
        linearized = np.concatenate((linearized, chunk), axis=0)
    return linearized


def detrend(feature, lat, lon, years, nombor=0):
    """
    Extract the mean feature value for each day in a year over the period between y1 and y2 inclusive and use
    it to do seasonal detrending.
    """
    y1, y2 = years
    data = None
    for y in range(y1, y2+1):
        more_data = get_daily_value_for_year(feature, lat, lon, y)
        if data is None:
            data = more_data
        else:
            data = np.concatenate((data, more_data), axis=0)
#    np.savetxt(f"daily_averages{nombor}.txt", data, delimiter=',')
    # means is an array where mean[0] is the feature value average over the years between y1 and y2 on the 
    # on the first day of a year
    means = np.array([np.array([data[i, j] for i in range(y2-y1+1)]).mean() for j in range(365)])
    detrended = data - means
    return detrended, means, data

def get_autummn_day_indices():
    autumn_1st_day = datetime(1979, 9, 22).timetuple().tm_yday
    autumn_last_day = datetime(1979, 12, 22).timetuple().tm_yday
    return (autumn_1st_day, autumn_last_day)

def get_winter_day_indices():
    winter_1st_day = datetime(1979, 12, 22).timetuple().tm_yday
    winter_last_day = datetime(1979, 3, 20).timetuple().tm_yday
    return (winter_1st_day, winter_last_day)

In [11]:
d = xr.open_dataset("E:\\weatherbench\\2.8125deg\\total_cloud_cover\\total_cloud_cover_2.8125deg\\total_cloud_cover_1979_2.8125deg.nc")
d

<xarray.Dataset>
Dimensions:  (lat: 64, lon: 128, time: 8760)
Coordinates:
  * lat      (lat) float64 -88.59 -85.78 -82.97 -80.16 ... 82.97 85.78 88.59
  * lon      (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
  * time     (time) datetime64[ns] 1979-01-01 ... 1979-12-31T23:00:00
Data variables:
    tcc      (time, lat, lon) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2020-01-01 13:38:30 GMT by grib_to_netcdf-2.15.0: /opt/ecmw...

In [18]:
semua = get_all_4_corners()
df = pd.read_csv("yearly_winter_anomaly_clusters.csv")

In [19]:
semua[0]

[(-88.59375, 0.0), (-88.59375, 2.8125), (-85.78125, 0.0), (-85.78125, 2.8125)]

lat:40.78125, lon:284.0625, 0
lat:40.78125, lon:286.875, 1
lat:43.59375, lon:284.0625, 2
lat:43.59375, lon:286.875, 3

In [20]:
semua = [[(40.78125, 284.0625), (40.78125, 286.875), (43.59375, 284.0625), (43.59375, 286.875)]]

In [17]:
#first_day, last_day = get_autummn_day_indices()
first_day, last_day = get_winter_day_indices()
avgs_empat = []
year_start = 1979
year_end = year_start + 38
if last_day < first_day:
    year_end += 1 # for winter data, data from one more year are needed
result_file = "sil_t2m_sanity_score.csv"
#result_file = "sil_t2m_score.csv"
with open(result_file, "a") as fout:
    fout.write("row,lat0,lon0,lat1,lon1,lat2,lon2,lat3,lon3,sil_score\n")
for i, one_empat in enumerate(semua):
    avgs_empat = []
    skip = False
    print(one_empat)
    for pt in one_empat:
        if pt[0] < 0.0:
            skip = True
            break
    if skip:
        continue
    for pt in one_empat:
        detrended, _, _ = detrend(("2m_temperature", 't2m'), *pt, (year_start, year_end))
        if last_day > first_day:
            avgs_empat.append([x.mean() for x in detrended[:, first_day:last_day]])
        else:
            front_chunk = detrended[:-1, first_day:]
            back_chunk = detrended[1:, :last_day] # 2nd part of winter is in the next year
            detrended = np.numpy((front_chunk, back_chunk), axis=1)
            avgs_empat.append([x.mean() for x in detrended])
    curr_score = metrics.silhouette_score(np.array(avgs_empat).T, df['cluster'])
    coords = ','.join([f"{pt[0]}, {pt[1]}" for pt in one_empat])
    result = f"{i},{coords},{curr_score}\n"
    with open(result_file, "a") as fout:
        fout.write(result)

(40.78125, 284.0625)


TypeError: 'float' object is not subscriptable